In [1]:
import json
import os
from glob import glob

import faiss
import numpy as np
import pandas as pd
import simplejson
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.std import tqdm

In [2]:
class Interactions(Dataset):
    def __init__(self, data: pd.DataFrame, track_features: pd.DataFrame):
        self.user = torch.as_tensor(data.user.values, dtype=torch.long)
        self.track_pos = torch.as_tensor(track_features.iloc[data.track_pos.values.squeeze()].values, dtype=torch.float32)
        self.track_neg = torch.as_tensor(track_features.iloc[data.track_neg.values.squeeze()].values, dtype=torch.float32)

    def __len__(self):
        return len(self.user)

    def __getitem__(self, idx):
        return self.user[idx], self.track_pos[idx], self.track_neg[idx]

In [3]:
class UserEncoder(nn.Module):
    def __init__(self, n_users: int, n_factors: int = 128, out_dim: int = 128):
        super().__init__()
        self.n_factors = n_factors
        self.user_embedding = nn.Embedding(n_users, n_factors)
        self.hidden = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Linear(n_factors, out_dim),
            nn.ReLU(inplace=True),
            nn.Linear(out_dim, out_dim),
        )

    def forward(self, user: torch.Tensor):
        user = user.reshape(-1)  # shape (batch_size,)
        user_emb = self.user_embedding(user)  # shape (batch_size, n_factors)
        hidden = self.hidden(user_emb)  # shape (batch_size, out_dim)
        return hidden


class ItemEncoder(nn.Module):
    def __init__(self, n_artists: int, n_genres: int, n_factors: int = 64, out_dim: int = 128):
        super().__init__()
        self.n_factors = n_factors
        self.artist_embedding = nn.Embedding(n_artists, n_factors)
        self.genre_embedding = nn.Embedding(n_genres, n_factors)
        self.pop_embedding = nn.Linear(1, n_factors)
        self.hidden = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Linear(n_factors * 3, out_dim),
            nn.ReLU(inplace=True),
            nn.Linear(out_dim, out_dim),
        )

    def forward(self, item: torch.Tensor):
        artist = item[:, 0].long().reshape(-1)  # shape (batch_size,)
        genre = item[:, 1].long().reshape(-1)  # shape (batch_size,)
        pop = item[:, 2].reshape(-1, 1)  # shape (batch_size, 1)
        artist_emb = self.artist_embedding(artist)  # shape (batch_size, n_factors)
        genre_emb = self.genre_embedding(genre)  # shape (batch_size, n_factors)
        pop_emb = self.pop_embedding(pop)  # shape (batch_size, n_factors)
        full_emb = torch.concat([artist_emb, genre_emb, pop_emb], dim=1)  # shape (batch_size, n_factors * 3)
        hidden = self.hidden(full_emb)  # shape (batch_size, out_dim)
        return hidden


class RecommenderModel(nn.Module):
    def __init__(self, n_users: int, n_artists: int, n_genres: int, encoder_out_dim: int = 100):
        super().__init__()
        self.user_encoder = UserEncoder(n_users, out_dim=encoder_out_dim)
        self.item_encoder = ItemEncoder(n_artists, n_genres, out_dim=encoder_out_dim)

    def forward(self, user: torch.Tensor, track_pos: torch.Tensor, track_neg: torch.Tensor):
        encoded_user = self.user_encoder(user)
        encoded_track_pos = self.item_encoder(track_pos)
        encoded_track_neg = self.item_encoder(track_neg)
        return encoded_user, encoded_track_pos, encoded_track_neg

In [4]:
track_features = (
    pd.read_json(os.path.join('..', 'sim', 'data', 'tracks.json'), lines=True)
    .drop(columns=['title'])
    .fillna('None')
    .sort_values('track')
    .drop(columns='track')
    .reset_index(drop=True)
)

artist_encoder = LabelEncoder().fit(track_features.artist)
track_features['artist'] = artist_encoder.transform(track_features.artist)
genre_encoder = LabelEncoder().fit(track_features.genre)
track_features['genre'] = genre_encoder.transform(track_features.genre)
track_features['pop'] = np.log(track_features['pop'].values) / 10

track_features

,artist,genre,pop
0,1266,14,1.096232
1,3881,14,1.087375
2,3262,5,1.091572
3,7838,15,0.748773
4,7838,15,0.880102
...,...,...,...
49995,10871,14,0.430407
49996,7734,16,0.400733
49997,370,14,0.412713
49998,3953,12,0.402535


In [5]:
train_dfs = []
for path in tqdm(
        glob(os.path.join('..', 'logs', 'log-*', 'data.json')),
        total=len(os.listdir(os.path.join('..', 'logs'))),
):
    with open(path, 'rt') as json_file:
        train_df = pd.DataFrame([simplejson.loads(line) for line in json_file])[
            ['user', 'track', 'time']
        ]
    train_dfs.append(train_df)
data = pd.concat(train_dfs).reset_index(drop=True)
print('Read all data!')

NUM_NEGATIVE_SAMPLES = 10
triplets = []
grouper = data.groupby('user')
for user, group in tqdm(grouper, total=len(grouper)):
    positives = group[group.time > 0.8].track.values
    negatives = group[group.time < 0.2].track.values

    expanded_positives = np.tile(positives, NUM_NEGATIVE_SAMPLES)
    sampled_negatives = np.random.choice(negatives, len(expanded_positives))
    user_expanded = np.full_like(sampled_negatives, user, dtype=int)

    user_triplets = pd.DataFrame(
        {
            'user': user_expanded,
            'track_pos': expanded_positives,
            'track_neg': sampled_negatives,
        }
    )
    triplets.append(user_triplets)
triplets = pd.concat(triplets).reset_index(drop=True)

train, val = train_test_split(triplets, stratify=triplets.user, test_size=0.1)
print(f'Train size: {len(train)}, val size: {len(val)}')
train

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [05:05<00:00,  7.27s/it]


Read all data!


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:17<00:00, 575.23it/s]


Train size: 13270959, val size: 1474551


,user,track_pos,track_neg
95798,68,4450,49594
11429963,7751,4529,36678
10476686,7117,45241,20646
10548174,7170,528,2332
14391083,9754,33871,18329
...,...,...,...
6900715,4710,41950,37152
10502264,7132,11744,32181
8862241,6032,41655,39651
1681636,1143,3283,41013


In [6]:
train_interactions = Interactions(train, track_features)
val_interactions = Interactions(val, track_features)

train_dataloader = DataLoader(
    train_interactions, batch_size=8192, shuffle=True, drop_last=True,
    num_workers=30, persistent_workers=True,
)
val_dataloader = DataLoader(
    val_interactions, batch_size=8192 * 16, shuffle=False, drop_last=False,
    num_workers=30, persistent_workers=True,
)

epochs = 40
# Initialize the model
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model = RecommenderModel(
    n_users=10000,
    n_artists=len(artist_encoder.classes_),
    n_genres=len(genre_encoder.classes_),
).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.TripletMarginLoss(margin=0.4)

In [7]:
# Train the model
for epoch in range(epochs):
    model.train()
    train_losses = []
    for user, track_pos, track_neg in tqdm(train_dataloader):
        user_, track_pos_, track_neg_ = map(
            lambda x: x.to(device), (user, track_pos, track_neg)
        )
        user_emb, track_pos_emb, track_neg_emb = model(user_, track_pos_, track_neg_)
        loss = criterion(user_emb, track_pos_emb, track_neg_emb)
        train_losses.append(loss.detach())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    for user, track_pos, track_neg in val_dataloader:
        user_, track_pos_, track_neg_ = map(
            lambda x: x.to(device), (user, track_pos, track_neg)
        )
        with torch.no_grad():
            user_emb, track_pos_emb, track_neg_emb = model(user_, track_pos_, track_neg_)
            loss = criterion(user_emb, track_pos_emb, track_neg_emb)
            val_losses.append(loss * len(user_))

    mean_train_loss = torch.stack(train_losses).mean().item()
    mean_val_loss = torch.stack(val_losses).sum().div(len(val_interactions)).item()
    print(f'Epoch {epoch}, train loss {mean_train_loss}, val loss {mean_val_loss}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1619/1619 [01:46<00:00, 15.20it/s]


Epoch 0, train loss 0.1511840522289276, val loss 0.1179153248667717


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1619/1619 [01:34<00:00, 17.18it/s]


Epoch 1, train loss 0.1035318374633789, val loss 0.09527994692325592


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1619/1619 [01:32<00:00, 17.54it/s]


...

Epoch 37, train loss 0.024676699191331863, val loss 0.03490293398499489


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1619/1619 [01:34<00:00, 17.05it/s]


Epoch 38, train loss 0.024388756603002548, val loss 0.034688930958509445


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1619/1619 [01:33<00:00, 17.28it/s]


Epoch 39, train loss 0.024111082777380943, val loss 0.034495025873184204


In [8]:
torch.save(model.state_dict(), 'model.pt')

In [9]:
model.eval()
with torch.no_grad():
    user_embeddings = model.user_encoder(torch.arange(0, 10000).to(device)).cpu().numpy()
    track_embeddings = model.item_encoder(
        torch.as_tensor(track_features.values, dtype=torch.float32).to(device)
    ).cpu().numpy()

In [10]:
track_embeddings.shape, user_embeddings.shape

((50000, 100), (10000, 100))

In [11]:
gpu_res = faiss.StandardGpuResources()
index_flat = faiss.index_factory(track_embeddings.shape[1], 'Flat', faiss.METRIC_L2)
index = faiss.index_cpu_to_gpu(gpu_res, 3, index_flat)
index.add(track_embeddings.astype('float32'))

In [12]:
with open(os.path.join('..', 'botify', 'data', 'recommendations_final.json'), 'wt') as rec_file:
    for user, user_emb in tqdm(enumerate(user_embeddings), total=len(user_embeddings)):
        dists, neighbours = index.search(user_emb.astype('float32')[np.newaxis, :], 30)
        recommendation = {
            "user": int(user),
            "tracks": neighbours.flatten().tolist()
        }
        rec_file.write(json.dumps(recommendation) + "\n")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 3306.46it/s]
